import os

# Directory containing JPEG images
jpeg_directory = r"C:\Users\net pc\Desktop\LIVER TUMOR\generated"

# Get the folder name
folder_name = os.path.basename(jpeg_directory)

# Initialize a counter for naming the images
counter = 1

# Iterate through JPEG files in the directory and rename them
for filename in os.listdir(jpeg_directory):
    if filename.endswith(".jpeg"):
        old_filepath = os.path.join(jpeg_directory, filename)
        new_filename = f"{folder_name}_{counter}.jpeg"
        new_filepath = os.path.join(jpeg_directory, new_filename)
        os.rename(old_filepath, new_filepath)
        counter += 1

print("Renaming complete.")


In [1]:

import tensorflow as tf
print(tf.__version__)


2.16.1


In [2]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade tensorflow


Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
print(tf.__version__)


2.16.1


In [5]:
pip install keras


Note: you may need to restart the kernel to use updated packages.


In [6]:
from keras.layers import Input


In [7]:
from tensorflow.keras.layers import Input
import tensorflow as tf
import pathlib
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, LeakyReLU, Activation, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from numpy import zeros, ones
from numpy.random import randint
from matplotlib import pyplot as plt


In [8]:
import numpy as np
import os
from skimage.transform import resize
from matplotlib import pyplot as plt

# Define file paths

input_dir = r"C:\Users\net pc\Desktop\LIVER TUMOR\malignant_input"
label_dir = r"C:\Users\net pc\Desktop\LIVER TUMOR\malignant_output"


def load_data(input_dir, label_dir, target_shape):
    input_files = os.listdir(input_dir)
    label_files = os.listdir(label_dir)
    
    # Sort files to ensure corresponding inputs and labels match
    input_files.sort()
    label_files.sort()
    
    trainA = []
    trainB = []
    for i in range(len(input_files)):
        input_img = plt.imread(os.path.join(input_dir, input_files[i]))
        label_img = plt.imread(os.path.join(label_dir, label_files[i]))
        
        # Resize images to target shape if they have different shapes
        if input_img.shape != target_shape:
            input_img = resize(input_img, target_shape)
        if label_img.shape != target_shape:
            label_img = resize(label_img, target_shape)
        
        trainA.append(input_img)
        trainB.append(label_img)
    
    return np.array(trainA), np.array(trainB)



# Define target shape for images (e.g., (256, 256, 3))
target_shape = (256, 256, 3)

# Load data
data = load_data(input_dir, label_dir, target_shape)
print("Data loaded successfully!")


Data loaded successfully!


In [9]:
# Define discriminator
def define_discriminator(image_shape):
    init = RandomNormal(stddev=0.02)
    in_src_image = Input(shape=image_shape, name='input_image')
    in_target_image = Input(shape=image_shape, name='target_image')
    merged = Concatenate()([in_src_image, in_target_image])
    
    # Discriminator architecture
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(patch_out)
    
    model = Model([in_src_image, in_target_image], patch_out)
    model.compile(loss='binary_crossentropy', optimizer='adam')
   
    return model


In [10]:
def define_generator(image_shape=(256, 256, 3)):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)
    
    # Encoder
    e1 = Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(in_image)
    e2 = Conv2D(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e1))
    e2 = BatchNormalization()(e2)
    e3 = Conv2D(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e2))
    e3 = BatchNormalization()(e3)
    e4 = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e3))
    e4 = BatchNormalization()(e4)
    e5 = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e4))
    e5 = BatchNormalization()(e5)
    e6 = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e5))
    e6 = BatchNormalization()(e6)
    e7 = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e6))
    e7 = BatchNormalization()(e7)
    
    # Bottleneck
    b = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e7))
    b = Activation('relu')(b)
    
    # Decoder
    d1 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(b))
    d1 = BatchNormalization()(d1)
    d1 = Dropout(0.5)(d1)
    d1 = Concatenate()([d1, e7])
    d2 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d1))
    d2 = BatchNormalization()(d2)
    d2 = Dropout(0.5)(d2)
    d2 = Concatenate()([d2, e6])
    d3 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d2))
    d3 = BatchNormalization()(d3)
    d3 = Concatenate()([d3, e5])
    d4 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d3))
    d4 = BatchNormalization()(d4)
    d4 = Concatenate()([d4, e4])
    d5 = Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d4))
    d5 = BatchNormalization()(d5)
    d5 = Concatenate()([d5, e3])
    d6 = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d5))
    d6 = BatchNormalization()(d6)
    d6 = Concatenate()([d6, e2])
    d7 = Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d6))
    d7 = BatchNormalization()(d7)
    d7 = Concatenate()([d7, e1])
    
    # Output
    g = Conv2DTranspose(image_shape[2], (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d7)
    out_image = Activation('tanh')(g)
    
    model = Model(in_image, out_image)
    return model


In [11]:
def generate_real_samples(dataset, n_samples, patch_shape):
    # unpack dataset
    trainA, trainB = dataset
    # choose random instances
    ix = randint(0, trainA.shape[0], n_samples)
    # retrieve selected images
    X1, X2 = trainA[ix], trainB[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return [X1, X2], y


In [12]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y


In [13]:
def define_gan(g_model, d_model, image_shape):
    # Make weights in the discriminator not trainable
    d_model.trainable = False
    
    # Input tensor for the generator model
    gen_input = Input(shape=image_shape)
    
    # Connect the generator output to the discriminator input
    gan_output = d_model([gen_input, g_model(gen_input)])
    
    # Define GAN model taking generator input and outputting discriminator decision
    gan_model = Model(gen_input, gan_output)
    
    # Compile GAN model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    gan_model.compile(loss=['binary_crossentropy'], optimizer=opt)
    
    return gan_model

# Define discriminator model
d_model = define_discriminator(target_shape)

# Define generator model
g_model = define_generator(target_shape)

# Define GAN model
gan_model = define_gan(g_model, d_model, target_shape)

C:\Users\Public\Efendim\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [14]:

import numpy as np

def train(d_model, g_model, gan_model, dataset, n_epochs=100, n_batch=1):
    # Determine the output square shape of the discriminator
    n_patch = d_model.output_shape[1]
    # Unpack dataset
    trainA, trainB = dataset
    # Calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)
    # Calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    
    for i in range(n_steps):
        # Select a batch of real samples
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        # Generate a batch of fake samples
        X_fakeB, _ = generate_fake_samples(g_model, X_realA, n_patch)
        # Update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], np.ones((n_batch, n_patch, n_patch, 1)))
        # Update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], np.zeros((n_batch, n_patch, n_patch, 1)))
        # Update the generator
        g_loss = gan_model.train_on_batch(X_realA, [np.ones((n_batch, n_patch, n_patch, 1)), X_realB])
        # Summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
        # Summarize model performance
        if (i+1) % (bat_per_epo * 10) == 0:
            summarize_performance(i, g_model, dataset)


In [15]:

# generate samples and save as a plot and save the model
#GAN models do not converge, we just want to find a good balance between
#the generator and the discriminator. Therefore, it makes sense to periodically
#save the generator model and check how good the generated image looks. 
import os
from PIL import Image

from PIL import Image


def save_img(filename, image):
    """Save the image to the specified filename."""
    # Scale the image back to the range [0, 255] before saving
    scaled_image = (image * 255).astype(np.uint8)
    # Convert to PIL Image
    im = Image.fromarray(scaled_image)
    # Save the image
    im.save(filename)
    
# Summarize performance
def summarize_performance(step, g_model, dataset, n_samples=3):
    print("Saving images...")
    # Select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    # Generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    # Scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
    

    output_path = "C:\\Users\\net pc\\Desktop\\LIVER TUMOR\\malignant_generated"



    for i in range(n_samples):
        # Save real source image as JPEG
        real_source_image = X_realA[i]
        real_source_filename = os.path.join(output_path, f'real_source_{step}_{i + 1}.jpeg')
        save_img(real_source_filename, real_source_image)

        # Save generated target image as JPEG
        generated_target_image = X_fakeB[i]
        generated_target_filename = os.path.join(output_path, f'generated_target_{step}_{i + 1}.jpeg')
        save_img(generated_target_filename, generated_target_image)

        # Save real target image as JPEG
        real_target_image = X_realB[i]
        real_target_filename = os.path.join(output_path, f'real_target_{step}_{i + 1}.jpeg')
        save_img(real_target_filename, real_target_image)

    # Save the generator model
    model_filename = os.path.join(output_path, f'model_{step}.h5')
    g_model.save(model_filename)

    print(f"Images and model saved at {output_path}")

    


In [16]:


# Train Pix2Pix models using the GAN model
train(d_model, g_model, gan_model, data, n_epochs=100, n_batch=1)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step


C:\Users\Public\Efendim\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:74: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


>1, d1[0.678] d2[0.690] g[0.687]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
>2, d1[0.687] d2[0.690] g[0.664]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>3, d1[0.688] d2[0.690] g[0.642]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
>4, d1[0.688] d2[0.690] g[0.624]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>5, d1[0.689] d2[0.690] g[0.609]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>6, d1[0.689] d2[0.691] g[0.595]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>7, d1[0.690] d2[0.691] g[0.583]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>8, d1[0.690] d2[0.691] g[0.572]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>9, d1[0.690] d2[0.691] g[0.562]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>10, d1[0.690] d2[0.691] g[0.553]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>11, d1[0.691] d2[0.691] g[0.545]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>12, d1[0.691] d2[0.691] g[0.537]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>13, d1[0.691] d2[0.691] g[0.530]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>14, d1[0.691] d2[0.692] g[0.524]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>

Images and model saved at C:\Users\net pc\Desktop\LIVER TUMOR\malignant_generated
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
>21, d1[0.692] d2[0.693] g[0.490]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
>22, d1[0.692] d2[0.693] g[0.486]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>23, d1[0.693] d2[0.693] g[0.483]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
>24, d1[0.693] d2[0.693] g[0.479]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
>25, d1[0.693] d2[0.693] g[0.476]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>26, d1[0.693] d2[0.694] g[0.473]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>27, d1[0.693] d2[0.694] g[0.470]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>28, d1[0.694] d2[0.694] g[0.467]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>29, d1[0.694] d2[0.694] g[0.464]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>30, d1[0.694] d2[0.695] g[0.462]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
>31, d1[0.694] d2[0.695] g[0.460]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>32, d1[0.695] d2[0.695] g[0.457]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
>33, d1[0.695] d

Images and model saved at C:\Users\net pc\Desktop\LIVER TUMOR\malignant_generated
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>41, d1[0.697] d2[0.698] g[0.439]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>42, d1[0.697] d2[0.698] g[0.437]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>43, d1[0.698] d2[0.698] g[0.436]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>44, d1[0.698] d2[0.698] g[0.434]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>45, d1[0.698] d2[0.699] g[0.432]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>46, d1[0.699] d2[0.699] g[0.431]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>47, d1[0.699] d2[0.699] g[0.429]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
>48, d1[0.699] d2[0.700] g[0.428]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
>49, d1[0.699] d2[0.700] g[0.426]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
>50, d1[0.700] d2[0.700] g[0.425]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
>51, d1[0.700] d2[0.701] g[0.424]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
>52, d1[0.700] d2[0.701] g[0.422]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>53, d1[0.701] d

...

Images and model saved at C:\Users\net pc\Desktop\LIVER TUMOR\malignant_generated
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>121, d1[0.730] d2[0.730] g[0.371]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
>122, d1[0.730] d2[0.731] g[0.371]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
>123, d1[0.731] d2[0.731] g[0.370]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
>124, d1[0.731] d2[0.732] g[0.370]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
>125, d1[0.732] d2[0.732] g[0.370]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
>126, d1[0.732] d2[0.733] g[0.369]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
>127, d1[0.733] d2[0.733] g[0.369]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>128, d1[0.733] d2[0.734] g[0.368]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>129, d1[0.734] d2[0.734] g[0.368]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
>130, d1[0.734] d2[0.735] g[0.368]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
>131, d1[0.735] d2[0.735] g[0.367]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
>132, d1[0.735] d2[0.736] g[0.367]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
>133

Images and model saved at C:\Users\net pc\Desktop\LIVER TUMOR\malignant_generated
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
>141, d1[0.740] d2[0.741] g[0.364]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
>142, d1[0.740] d2[0.741] g[0.363]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
>143, d1[0.741] d2[0.742] g[0.363]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
>144, d1[0.741] d2[0.742] g[0.363]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
>145, d1[0.742] d2[0.743] g[0.362]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
>146, d1[0.743] d2[0.743] g[0.362]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
>147, d1[0.743] d2[0.744] g[0.362]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
>148, d1[0.744] d2[0.744] g[0.361]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
>149, d1[0.744] d2[0.745] g[0.361]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
>150, d1[0.745] d2[0.745] g[0.361]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
>151, d1[0.745] d2[0.746] g[0.360]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
>152, d1[0.746] d2[0.747] g[0.360]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
>153

Images and model saved at C:\Users\net pc\Desktop\LIVER TUMOR\malignant_generated
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
>161, d1[0.751] d2[0.751] g[0.357]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
>162, d1[0.751] d2[0.752] g[0.357]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
>163, d1[0.752] d2[0.753] g[0.357]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
>164, d1[0.752] d2[0.753] g[0.356]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
>165, d1[0.753] d2[0.754] g[0.356]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
>166, d1[0.753] d2[0.754] g[0.356]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>167, d1[0.754] d2[0.755] g[0.355]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
>168, d1[0.755] d2[0.755] g[0.355]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
>169, d1[0.755] d2[0.756] g[0.355]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
>170, d1[0.756] d2[0.756] g[0.355]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
>171, d1[0.756] d2[0.757] g[0.354]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
>172, d1[0.757] d2[0.758] g[0.354]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
>173

Images and model saved at C:\Users\net pc\Desktop\LIVER TUMOR\malignant_generated
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
>181, d1[0.762] d2[0.763] g[0.352]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
>182, d1[0.762] d2[0.763] g[0.351]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
>183, d1[0.763] d2[0.764] g[0.351]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
>184, d1[0.763] d2[0.764] g[0.351]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
>185, d1[0.764] d2[0.765] g[0.351]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
>186, d1[0.764] d2[0.765] g[0.350]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
>187, d1[0.765] d2[0.766] g[0.350]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
>188, d1[0.766] d2[0.766] g[0.350]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
>189, d1[0.766] d2[0.767] g[0.350]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
>190, d1[0.767] d2[0.768] g[0.349]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
>191, d1[0.767] d2[0.768] g[0.349]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
>192, d1[0.768] d2[0.769] g[0.349]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
>193

Images and model saved at C:\Users\net pc\Desktop\LIVER TUMOR\malignant_generated
